In [1]:
pip install sounddevice scipy librosa

Note: you may need to restart the kernel to use updated packages.


In [45]:
import sounddevice as sd
import numpy as np
import librosa
import joblib
from scipy.io.wavfile import write

# -----------------------------
# Load trained model & encoder
# -----------------------------
# rf_model = joblib.load("random_forest_chord_model.pkl")
# label_encoder = joblib.load("label_encoder.pkl")

rf_model = joblib.load("random_forest_chord_model.pk1")
label_encoder = joblib.load("label_encoder.pk1")
# -----------------------------
# Audio recording parameters
# -----------------------------
duration = 2        # seconds (IMPORTANT)
sample_rate = 22050

print("🎤 Recording... Play a chord now")
audio = sd.rec(
    int(duration * sample_rate),
    samplerate=sample_rate,
    channels=1
)
sd.wait()
print("✅ Recording finished")

# Save recorded audio
write("mic_input.wav", sample_rate, audio)

# -----------------------------
# Feature extraction
# -----------------------------
def extract_chroma(file_path):
    y, sr = librosa.load(file_path, sr=22050, mono=True)
    y, _ = librosa.effects.trim(y, top_db=20)
    y = librosa.util.normalize(y)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    return np.mean(chroma, axis=1)

chroma = extract_chroma("mic_input.wav")
chroma = chroma.reshape(1, -1)

# -----------------------------
# Prediction
# -----------------------------
prediction = rf_model.predict(chroma)
predicted_chord = label_encoder.inverse_transform(prediction)

print("🎹 Predicted Chord:", predicted_chord[0])


🎤 Recording... Play a chord now
✅ Recording finished
🎹 Predicted Chord: C


/opt/anaconda3/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
